# Simpson's Rule for Numerical Integration

## Introduction

Numerical integration is a fundamental technique in computational mathematics for approximating definite integrals when analytical solutions are difficult or impossible to obtain. **Simpson's Rule** is one of the most widely used methods due to its excellent accuracy-to-computation ratio.

## Theoretical Foundation

### The Problem Statement

Given a continuous function $f(x)$ on the interval $[a, b]$, we seek to approximate the definite integral:

$$I = \int_a^b f(x) \, dx$$

### Derivation of Simpson's Rule

Simpson's Rule is based on approximating the integrand $f(x)$ by a quadratic polynomial that passes through three equally spaced points. Consider the interval $[x_0, x_2]$ with midpoint $x_1 = \frac{x_0 + x_2}{2}$ and spacing $h = \frac{x_2 - x_0}{2}$.

The Lagrange interpolating polynomial of degree 2 through the points $(x_0, f_0)$, $(x_1, f_1)$, and $(x_2, f_2)$ is:

$$P_2(x) = f_0 \frac{(x - x_1)(x - x_2)}{(x_0 - x_1)(x_0 - x_2)} + f_1 \frac{(x - x_0)(x - x_2)}{(x_1 - x_0)(x_1 - x_2)} + f_2 \frac{(x - x_0)(x - x_1)}{(x_2 - x_0)(x_2 - x_1)}$$

Integrating this polynomial over $[x_0, x_2]$ yields:

$$\int_{x_0}^{x_2} P_2(x) \, dx = \frac{h}{3}(f_0 + 4f_1 + f_2)$$

This is the **basic Simpson's Rule** for a single parabolic segment.

### Composite Simpson's Rule

For better accuracy, we divide $[a, b]$ into $n$ subintervals (where $n$ must be **even**) with spacing $h = \frac{b - a}{n}$. The composite formula becomes:

$$\int_a^b f(x) \, dx \approx \frac{h}{3} \left[ f(x_0) + 4\sum_{i=1,3,5,...}^{n-1} f(x_i) + 2\sum_{i=2,4,6,...}^{n-2} f(x_i) + f(x_n) \right]$$

Or more compactly:

$$S_n = \frac{h}{3} \left[ f_0 + 4f_1 + 2f_2 + 4f_3 + 2f_4 + \cdots + 4f_{n-1} + f_n \right]$$

### Error Analysis

The error in Simpson's Rule is:

$$E = -\frac{(b-a)^5}{180n^4} f^{(4)}(\xi)$$

for some $\xi \in [a, b]$. This shows that Simpson's Rule has **fourth-order accuracy** $O(h^4)$, making it exact for polynomials up to degree 3.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate

# Set up plotting style
plt.rcParams['figure.figsize'] = [12, 10]
plt.rcParams['font.size'] = 11
plt.rcParams['axes.grid'] = True

## Implementation of Simpson's Rule

We implement the composite Simpson's Rule algorithm from first principles.

In [ ]:
def simpsons_rule(f, a, b, n):
    """
    Approximate the integral of f from a to b using Simpson's Rule.
    
    Parameters:
    -----------
    f : callable
        Function to integrate
    a : float
        Lower limit of integration
    b : float
        Upper limit of integration
    n : int
        Number of subintervals (must be even)
    
    Returns:
    --------
    float
        Approximate value of the integral
    """
    if n % 2 != 0:
        raise ValueError("n must be even for Simpson's Rule")
    
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = f(x)
    
    # Simpson's Rule: (h/3) * [f0 + 4*f1 + 2*f2 + 4*f3 + ... + 4*f_{n-1} + fn]
    integral = y[0] + y[-1]  # Endpoints
    integral += 4 * np.sum(y[1:-1:2])  # Odd indices (coefficient 4)
    integral += 2 * np.sum(y[2:-1:2])  # Even indices (coefficient 2)
    integral *= h / 3
    
    return integral

## Test Case 1: Gaussian Integral

We test Simpson's Rule on the Gaussian function $f(x) = e^{-x^2}$. The integral:

$$\int_0^1 e^{-x^2} \, dx \approx 0.7468241328$$

is known to high precision.

In [ ]:
# Define test function
def gaussian(x):
    return np.exp(-x**2)

# Integration limits
a, b = 0, 1

# Exact value (computed with high-precision quadrature)
exact_value, _ = integrate.quad(gaussian, a, b)
print(f"Reference value (scipy.quad): {exact_value:.10f}")

# Test with different numbers of subintervals
n_values = [2, 4, 8, 16, 32, 64, 128, 256]
errors = []

print("\nSimpson's Rule Results:")
print("-" * 50)
print(f"{'n':>6} {'Approximation':>18} {'Error':>15}")
print("-" * 50)

for n in n_values:
    approx = simpsons_rule(gaussian, a, b, n)
    error = abs(approx - exact_value)
    errors.append(error)
    print(f"{n:>6} {approx:>18.10f} {error:>15.2e}")

## Convergence Analysis

We verify the fourth-order convergence rate $O(h^4)$ by plotting the error versus the step size $h$.

In [ ]:
# Calculate step sizes
h_values = [(b - a) / n for n in n_values]

# Compute convergence rate
log_h = np.log10(h_values)
log_err = np.log10(errors)

# Linear regression to find slope (convergence order)
coeffs = np.polyfit(log_h[2:], log_err[2:], 1)  # Skip first points for cleaner fit
slope = coeffs[0]

print(f"\nEmpirical convergence order: {slope:.2f}")
print(f"Expected convergence order: 4.00")

## Test Case 2: Oscillatory Function

We test on a more challenging oscillatory function:

$$f(x) = \sin(x^2)$$

over $[0, \sqrt{\pi}]$. This integral is related to the Fresnel integral.

In [ ]:
def oscillatory(x):
    return np.sin(x**2)

a2, b2 = 0, np.sqrt(np.pi)
exact_osc, _ = integrate.quad(oscillatory, a2, b2)

print(f"Oscillatory function: sin(x²)")
print(f"Integration domain: [0, √π]")
print(f"Reference value: {exact_osc:.10f}")

errors_osc = []
print("\n" + "-" * 50)
print(f"{'n':>6} {'Approximation':>18} {'Error':>15}")
print("-" * 50)

for n in n_values:
    approx = simpsons_rule(oscillatory, a2, b2, n)
    error = abs(approx - exact_osc)
    errors_osc.append(error)
    print(f"{n:>6} {approx:>18.10f} {error:>15.2e}")

## Comparison with Trapezoidal Rule

To appreciate Simpson's Rule's superior accuracy, we compare it with the Trapezoidal Rule, which has only $O(h^2)$ convergence.

In [ ]:
def trapezoidal_rule(f, a, b, n):
    """
    Approximate the integral using the Trapezoidal Rule.
    """
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = f(x)
    
    integral = (y[0] + y[-1]) / 2 + np.sum(y[1:-1])
    integral *= h
    
    return integral

# Compare errors for Gaussian integral
errors_trap = []
for n in n_values:
    approx_trap = trapezoidal_rule(gaussian, a, b, n)
    error_trap = abs(approx_trap - exact_value)
    errors_trap.append(error_trap)

print("Comparison: Simpson's Rule vs Trapezoidal Rule")
print("-" * 60)
print(f"{'n':>6} {'Simpson Error':>18} {'Trapezoid Error':>18}")
print("-" * 60)
for i, n in enumerate(n_values):
    print(f"{n:>6} {errors[i]:>18.2e} {errors_trap[i]:>18.2e}")

## Visualization

We create a comprehensive visualization showing:
1. The parabolic approximation used by Simpson's Rule
2. Convergence comparison between methods
3. Error analysis for different test functions

In [ ]:
fig = plt.figure(figsize=(14, 10))

# Plot 1: Visual representation of Simpson's Rule
ax1 = fig.add_subplot(221)

x_fine = np.linspace(a, b, 1000)
ax1.plot(x_fine, gaussian(x_fine), 'b-', linewidth=2, label=r'$f(x) = e^{-x^2}$')

# Show Simpson's rule with n=4 (2 parabolic segments)
n_demo = 4
x_pts = np.linspace(a, b, n_demo + 1)
y_pts = gaussian(x_pts)

# Plot the parabolic approximations
for i in range(0, n_demo, 2):
    x_seg = np.linspace(x_pts[i], x_pts[i+2], 50)
    # Lagrange interpolation through 3 points
    x0, x1, x2 = x_pts[i], x_pts[i+1], x_pts[i+2]
    y0, y1, y2 = y_pts[i], y_pts[i+1], y_pts[i+2]
    
    L0 = (x_seg - x1) * (x_seg - x2) / ((x0 - x1) * (x0 - x2))
    L1 = (x_seg - x0) * (x_seg - x2) / ((x1 - x0) * (x1 - x2))
    L2 = (x_seg - x0) * (x_seg - x1) / ((x2 - x0) * (x2 - x1))
    
    parabola = y0 * L0 + y1 * L1 + y2 * L2
    ax1.fill_between(x_seg, 0, parabola, alpha=0.3, color='orange')
    ax1.plot(x_seg, parabola, 'r--', linewidth=1.5)

ax1.plot(x_pts, y_pts, 'ko', markersize=8, label='Sample points')
ax1.set_xlabel('x')
ax1.set_ylabel('f(x)')
ax1.set_title("Simpson's Rule: Parabolic Approximation (n=4)")
ax1.legend()
ax1.set_xlim([a - 0.1, b + 0.1])

# Plot 2: Convergence comparison
ax2 = fig.add_subplot(222)

ax2.loglog(h_values, errors, 'bo-', linewidth=2, markersize=8, label="Simpson's Rule")
ax2.loglog(h_values, errors_trap, 'rs-', linewidth=2, markersize=8, label='Trapezoidal Rule')

# Reference lines for O(h^2) and O(h^4)
h_ref = np.array(h_values)
ax2.loglog(h_ref, 0.1 * h_ref**2, 'g--', alpha=0.7, linewidth=1.5, label=r'$O(h^2)$')
ax2.loglog(h_ref, 0.1 * h_ref**4, 'm--', alpha=0.7, linewidth=1.5, label=r'$O(h^4)$')

ax2.set_xlabel('Step size h')
ax2.set_ylabel('Absolute Error')
ax2.set_title('Convergence Comparison: Gaussian Integral')
ax2.legend(loc='lower right')
ax2.grid(True, which='both', alpha=0.3)

# Plot 3: Oscillatory function and its integral
ax3 = fig.add_subplot(223)

x_osc = np.linspace(a2, b2, 1000)
ax3.plot(x_osc, oscillatory(x_osc), 'b-', linewidth=2, label=r'$f(x) = \sin(x^2)$')
ax3.fill_between(x_osc, 0, oscillatory(x_osc), alpha=0.3, color='blue')
ax3.axhline(y=0, color='k', linewidth=0.5)

ax3.set_xlabel('x')
ax3.set_ylabel('f(x)')
ax3.set_title(f'Oscillatory Function (Integral ≈ {exact_osc:.4f})')
ax3.legend()

# Plot 4: Error comparison for both test functions
ax4 = fig.add_subplot(224)

ax4.loglog(n_values, errors, 'bo-', linewidth=2, markersize=8, label='Gaussian')
ax4.loglog(n_values, errors_osc, 'g^-', linewidth=2, markersize=8, label='Oscillatory')

ax4.set_xlabel('Number of subintervals n')
ax4.set_ylabel('Absolute Error')
ax4.set_title("Simpson's Rule: Error vs Number of Subintervals")
ax4.legend()
ax4.grid(True, which='both', alpha=0.3)

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nPlot saved to 'plot.png'")

## Summary

### Key Results

1. **Simpson's Rule** achieves $O(h^4)$ convergence, making it significantly more accurate than the Trapezoidal Rule's $O(h^2)$ for smooth functions.

2. The method is exact for polynomials up to degree 3, which explains its efficiency for well-behaved functions.

3. For the Gaussian integral with $n = 256$ subintervals, Simpson's Rule achieves errors on the order of $10^{-15}$ (machine precision), while the Trapezoidal Rule has errors around $10^{-7}$.

### Practical Considerations

- **Even $n$ requirement**: Simpson's Rule requires an even number of subintervals.
- **Smoothness**: The method works best for functions with continuous fourth derivatives.
- **Oscillatory functions**: May require more subintervals to capture rapid oscillations.

### Extensions

- **Adaptive Simpson's Rule**: Automatically refines mesh where error is large
- **Simpson's 3/8 Rule**: Uses cubic interpolation for slightly different weighting
- **Romberg Integration**: Combines multiple Simpson estimates for even higher accuracy